## DELTA_DIST+H_05 - Recherche par quadrillage des hyper-paramètres

Maintenant que nous avons un modèle très rapide à entraîner (voir notebook 8.3), nous allons pouvoir lancer une recherche par quadrillage approfondie des hyper-paramètres. Nous allons pour cela utiliser la classe GridSearchCV de Scikit-Learn, qui permet d'automatiser simplement la recherche par quadrillage avec validation croisée.

Pour utiliser Scikit-Learn, nous allons devoir écrire une classe permettant d'utiliser notre modèle basé sur TFLearn comme un modèle Scikit-Learn.

#### Chemin des fichiers

In [1]:
train_prepared_input_loc = "../data/train_set_riken_v2_prepared_input_bruit+_anums_reduced.h5"
train_labels_loc = "../data/train_set_riken_v2_labels_bruit+_anums_reduced.h5"

mini_prepared_input_loc = "../data/mini_set_prepared_input_bruit+_anums_reduced.h5"
mini_labels_loc = "../data/mini_set_labels_bruit+_anums_reduced.h5"

models_loc = "../models/DELTA_DIST+H_05/8.4-GridSearchCV/"
logs_loc = "../models/DELTA_DIST+H_05/8.4-GridSearchCV/"

## Création du RN

#### Fonction renvoyant le masque des atomes à prendre en compte pour les calculs

L'entrée et la sortie du RN étant définies par une méthode de padding, seul un certain nombre d'entrées et de sortie est utilisé pour chaque exemple d'entraînement en fonction du nombre d'atomes de la molécule. On définit ici une fonction qui renvoie le masque des différences de distances à prendre en compte sur les données en entrée et les étiquettes.

In [2]:
import tensorflow as tf

def calcul_masque_atomes_definis(targets):
    """ On calcule le masque booléen des atomes donnés en entrée du RN en fonction du vecteur targets"""
    
    # On cherche à obtenir un masque booléen des atomes définis en entrée. Pour cela, on prend en entrée
    # les étiquettes sous la forme d'une matrice (200, 4) dont chaque ligne i est la distance de l'atome i avec
    # les atomes fictifs du repère. L'atome est indéfini ssi. la somme de la ligne est nulle. En effet,
    # un atome défini ne peut pas avoir une distance nulle avec les quatre atomes fictifs, et on veille
    # à ce que le vecteurs targets ne contienne que des valeurs nulles pour les atomes non définis.
    # On obtient donc un masque booléen de tous les atomes définis en entrée
    
    ## On somme les distances de chaque atome ##
    targets_dists_sums = tf.reduce_sum(targets, 1)
    
    ## On créé le masque des sommes différentes de zéro ##
    
    # Création des matrice de True et de False de la dimension de la matrice des sommes (nécessaires
    # pour tf.where)
    zeros = tf.cast(tf.zeros_like(targets_dists_sums),dtype=tf.bool)
    ones = tf.cast(tf.ones_like(targets_dists_sums),dtype=tf.bool)
    
    return tf.where(targets_dists_sums>0, ones, zeros)


/home/etudiant/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### Fonction de coût


In [4]:
def partial_rmse(predictions, targets):
    """ Calcule le RMSE partiel des prédictions par rapport aux valeurs attendues. Le RMSE est partiel car
    on ne le calcule que pour les sorties correspondant aux atomes donnés en entrée. En d'autres
    termes, on ne pousse pas le modèle à donner des distances nulles pour les atomes indéfinis
    en entrée"""
    
    with tf.name_scope("partial_rmse"):

        # On met les prédictions et les cibles sous la forme d'une matrice (200, 4)
        predictions = tf.reshape(predictions, [-1, 4])
        targets = tf.reshape(targets, [-1, 4])

        # On calcule le masque des atomes définis selon les cibles
        defined_atoms_mask = calcul_masque_atomes_definis(targets)
        
        # On masque les prédictions et les étiquettes selon le masque des atomes définis
        targets_masked = tf.boolean_mask(targets, defined_atoms_mask)
        predictions_masked = tf.boolean_mask(predictions, defined_atoms_mask)   

        return tf.sqrt(tf.reduce_mean(tf.squared_difference(predictions_masked, targets_masked)), name="rmse")
    

#### Fonction d'évaluation des performances (score R2)

In [5]:
def partial_r2_score(predictions, targets, inputs):
    """ Renvoie le score R2 de la prédiction (le calcul est effectué uniquement sur les résultats
    des atomes donnés en entrée) """
    
    with tf.name_scope("partial_r2"):
    
        # On met les prédictions et les cibles sous la forme d'une matrice (200, 4)
        predictions = tf.reshape(predictions, [-1, 4])
        targets = tf.reshape(targets, [-1, 4])

        # On calcule le masque des atomes définis selon les cibles
        defined_atoms_mask = calcul_masque_atomes_definis(targets)

        # On masque les prédictions et les étiquettes selon le masque des atomes définis
        targets_masked = tf.boolean_mask(targets, defined_atoms_mask)
        predictions_masked = tf.boolean_mask(predictions, defined_atoms_mask)

        # Calcul de l'erreur totale
        total_error = tf.reduce_sum(tf.square(tf.subtract(targets, tf.reduce_mean(targets_masked))))

        # Calcul de l'erreur inexpliquée
        unexplained_error = tf.reduce_sum(tf.square(tf.subtract(targets_masked, predictions_masked)))

        r2 = tf.subtract(1.0, tf.divide(unexplained_error, total_error), "r2_score")
        return r2

#### Définition d'une fonction créant le RN

In [6]:
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.optimizers import Adam
from tflearn.data_preprocessing import DataPreprocessing
import math


def creer_360_x_240(epsilon=1e-8, learning_rate=0.001, dropout_val=0.99, stddev_init=0.001,
                      hidden_act='relu', outlayer_act='prelu', weight_decay=0.001, width=360, depth=3):
    """ Fonction créant un réseau de neurones de type fully connected, ayant une couche d'entrée de 360
    neurones, quatre couches cachées de 360 neurones et une sortie de 240 neurones
    Inputs : hyperparamètres
    """

    # On créé l'initialisateur de tenseur avec une loi normale tronquée. sigma = stddev_init, et les 
    # valeurs à plus de 2sigma sont re-tirées
    winit = tfl.initializations.truncated_normal(stddev=stddev_init, dtype=tf.float32, seed=None)
    
    # On créé l'input du RN
    network = input_data(shape=[None, 360], name='input')
    
    # On créé les couches cachées
    for i in range(depth):
        network = fully_connected(network, width, activation=hidden_act, name='fc'+str(i), weights_init=winit,
                                  weight_decay=weight_decay)
        # On détruit des neurones aléatoirement avec une la probabilité donnée en entrée
        network = dropout(network, dropout_val)
    
    # On ajoute la couche de sortie du réseau
    # Fonction d'activation prelu
    # Initilisée avec la loi normale tronquée
    network = fully_connected(network, 240, activation=outlayer_act, name='outlayer', weights_init=winit)
    
    adam = Adam(learning_rate=learning_rate, epsilon=epsilon)
    
    # Couche d'évaluation du modèle. Utilisation d'une descente stochastique Adam
    # Learning rate = 0.05
    # Loss = fonction définie rmse
    network = regression(network, optimizer=adam,
    loss=partial_rmse, metric=partial_r2_score, name='target')
            
    return network

Instructions for updating:
Use the retry module or similar alternatives.


## Préparation des données


#### Fonction renvoyant deux sous-ensembles du jeu d'entrainement : un ensemble d'exemples et les cibles correspondantes


In [8]:
def get_fold(train_set, targets, reduce_train_fold_size):
    """ Permet d'obtenir un sous-ensemble du jeu d'entraînement afin de ne pas travailler sur le jeu
    d'entraînement total pour la recherche par quadrillage et donc de gagner du temps d'exécution. L'idée
    et que si un ensemble d'hyper-paramètres produit des meilleurs résultats que les autres ensembles
    d'hyper-paramètres sur l'ensemble du jeu d'entraînement, alors on suppose que ce sera également 
    le cas sur une partie des données. """

    return (train_set["inputs"][:reduce_train_fold_size], targets["targets"][:reduce_train_fold_size])

## Entraînement des modèles

#### Classe permettant d'interfacer les modèles TFLearn avec Scikit-Learn

In [1]:
from sklearn.base import BaseEstimator, RegressorMixin
import tensorflow as tf
import tflearn as tfl



class RNRegressor(BaseEstimator, RegressorMixin):
    """ Wrapper autour de tflearn que l'on utilise pour utiliser les modèles depuis Scikit-Learn
    Inspiré de http://danielhnyk.cz/creating-your-own-estimator-scikit-learn/ """
    
    
    def __init__(self, log_dir="../models/logs" ,learning_rate=0.001, epsilon=1e-8, dropout=0.99, 
                 stddev_init=0.001, hidden_act="relu", outlayer_act='linear', weight_decay=0.001, 
                 width=360, depth=3, batch_size=10000, epochs=1):
        
        self.log_dir = log_dir
        self.learning_rate = learning_rate
        self.epsilon = epsilon
        self.dropout = dropout
        self.stddev_init = stddev_init
        self.hidden_act = hidden_act
        self.outlayer_act = outlayer_act
        self.weight_decay = weight_decay
        self.width = width
        self.depth = depth
        self.batch_size = batch_size
        self.epochs = epochs
        
        
    def fit(self, X, y=None):
        
        tf.reset_default_graph()

        # On créé le réseau 
        network = creer_360_x_240(learning_rate=self.learning_rate, epsilon=self.epsilon, 
                                      dropout_val=self.dropout, stddev_init=self.stddev_init, 
                                      hidden_act=self.hidden_act, outlayer_act=self.outlayer_act,
                                      weight_decay=self.weight_decay, width=self.width, depth=self.depth)

        # On créé le modèle
        self.model = tfl.DNN(network, tensorboard_verbose=3, tensorboard_dir=logs_path)

        model_name = ("lr"+str(self.learning_rate)+"eps"+str(self.epsilon)+"do"+stddevinit+str(self.stddev_init)+
                     "hidact"+self.hidden_act+"out"+self.outlayer_act+"wd"+str(self.weight_decay)+
                     "w"+str(self.width)+"d"+str(self.depth)+"batchs"+str(self.batch_size)+"epochs"+str(epochs))
        
        # Entraînement
        self.model.fit(X_inputs=X,Y_targets=y, batch_size=self.batch_size,
                  shuffle = False, snapshot_step=1000, validation_set=0,
                  show_metric=True, run_id=model_name, n_epoch=self.epochs)
        
        
        
    def predict(self, X, y=None):
        return self.model.predict(X, y)
    
    
    def score(self, X, y=None):
        return self.model.evaluate(X, y, batch_size=1000)

    
    

/home/etudiant/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.
